#1.1 직접구현

In [61]:
docs = ['나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해',
        '나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해']

1) 띄어쓰기 단위로 토큰화

In [62]:
doc_ls = [i.split() for i in docs]
doc_ls

[['나는', '양념', '치킨을', '좋아해', '하지만', '후라이드', '치킨을', '싫어해'],
 ['나는', '후라이드', '치킨을', '좋아해', '하지만', '양념', '치킨을', '싫어해']]

2) 각 고유 토큰에 인덱스(index)를 지정

In [63]:
from collections import defaultdict

word2id = defaultdict(lambda : len(word2id))
[word2id[token] for doc in doc_ls for token in doc]
word2id

defaultdict(<function __main__.<lambda>>,
            {'나는': 0,
             '싫어해': 6,
             '양념': 1,
             '좋아해': 3,
             '치킨을': 2,
             '하지만': 4,
             '후라이드': 5})

3) BoW 생성

In [64]:
import numpy as np

BoW_ls = []

for i, doc in enumerate(doc_ls):
  bow = np.zeros(len(word2id), dtype=int)
  for token in doc:
    bow[word2id[token]] +=1  # 해당 토큰의 위치(column)
  BoW_ls.append(bow.tolist())

BoW_ls

[[1, 1, 2, 1, 1, 1, 1], [1, 1, 2, 1, 1, 1, 1]]

In [65]:
from IPython.core import display as ICD

sorted_vocab = sorted((value, key) for key, value in word2id.items())
vocab = [v[1] for v in sorted_vocab]
for i in range(len(docs)) :
  print("문서{} : {}".format(i, docs[i]))
  ICD.display(pd.DataFrame([BoW_ls[i]], columns=vocab))
  print("\n\n")

문서0 : 나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,1,1,1,1





문서1 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,1,1,1,1


#1.3 sklearn 활용

In [66]:
docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

In [68]:
CountVectorizer?

In [69]:
# bag of word를 만들어내는 sklearn 코드
count_vect = CountVectorizer()
BoW = count_vect.fit_transform(docs)

BoW.toarray()[0]

array([1, 0, 1, 1, 1, 0, 0, 0])

In [70]:
import pandas as pd

In [71]:
from IPython.core import display as ICD

vocab = count_vect.get_feature_names()
for i in range(len(docs)) :
  print("문서{} : {}".format(i, docs[i]))
  ICD.display(pd.DataFrame([BoW.toarray()[i]], columns=vocab))
  print("\n\n")

#ICD는 pandas dataframe 받음

문서0 : 오늘 동물원에서 원숭이를 봤어


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,0,1,1,1,0,0,0





문서1 : 오늘 동물원에서 코끼리를 봤어 봤어


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,0,2,1,0,0,0,1





문서2 : 동물원에서 원숭이에게 바나나를 줬어 바나나를


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,2,0,0,0,1,1,0


In [72]:
!pip install gensim

#1.4 gensim 활용

In [29]:
import gensim
from gensim import corpora

In [30]:
docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [32]:
doc_ls = [doc.split() for doc in docs]
id2word = corpora.Dictionary(doc_ls)
BoW = [id2word.doc2bow(doc) for doc in doc_ls]

In [33]:
BoW[0]

[(0, 1), (1, 1), (2, 1), (3, 1)]

In [53]:
from gensim.matutils import sparse2full
from IPython.core import display as ICD

vocab = [id2word[i] for i in id2word.keys()]
for i in range(len(docs)) :
  print("문서{} : {}".format(i, docs[i]))
  ICD.display(pd.DataFrame([sparse2full(BoW[i], len(vocab))], columns=vocab))
  print("\n\n")
         # vocab = [id2word.keys()]

문서0 : 오늘 동물원에서 원숭이를 봤어


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0





문서1 : 오늘 동물원에서 코끼리를 봤어 봤어


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0





문서2 : 동물원에서 원숭이에게 바나나를 줬어 바나나를


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


In [38]:
id2word[0]

'동물원에서'

# 2. TDM(Term_Document Matrix) 직접구현

In [ ]:
docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [47]:
doc_ls = [doc.split() for doc in docs]
doc_ls

[['오늘', '동물원에서', '원숭이를', '봤어'],
 ['오늘', '동물원에서', '코끼리를', '봤어', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [48]:
from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))
[word2id[token] for doc in doc_ls for token in doc]
word2id

defaultdict(<function __main__.<lambda>>,
            {'동물원에서': 1,
             '바나나를': 6,
             '봤어': 3,
             '오늘': 0,
             '원숭이를': 2,
             '원숭이에게': 5,
             '줬어': 7,
             '코끼리를': 4})

In [52]:
import numpy as np
TDM = np.zeros((len(word2id), len(doc_ls)), dtype=int)
for i, doc in enumerate(doc_ls):
  for token in doc:
    TDM[word2id[token], i] += 1
TDM

array([[1, 1, 0],
       [1, 1, 1],
       [1, 0, 0],
       [1, 2, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 1]])

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
DTM = count_vect.fit_transform(docs)
DTM.T.toarray()

array([[1, 1, 1],
       [0, 0, 2],
       [1, 2, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]])

In [57]:
import pandas as pd

doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = count_vect.get_feature_names()
df_TDM = pd.DataFrame(DTM.T.toarray(), columns=doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2
단어,,,
동물원에서,1,1,1
바나나를,0,0,2
봤어,1,2,0
오늘,1,1,0
원숭이를,1,0,0
원숭이에게,0,0,1
줬어,0,0,1
코끼리를,0,1,0
